In [3]:
# --- deps: pip install filterpy sympy numpy ---
import numpy as np
import sympy as sp

In [4]:
px, py, pz, vx, vy, vz, qw, qx, qy, qz = sp.symbols('p_{x} p_{y} p_{z} v_{x} v_{y} v_{z} q_{w} q_{x} q_{y} q_{z}')
bwx, bwy, bwz, bax, bay, baz =  sp.symbols('b_{wx} b_{wy} b_{wz} b_{ax} b_{ay} b_{az}')

In [5]:
x =sp.Matrix([px, py, pz, vx, vy, vz, qw, qx, qy, qz, bwx, bwy, bwz, bax, bay, baz])

In [6]:
p = sp.Matrix([
    px, py, pz
])

v = sp.Matrix([
    vx, vy, vz
])
q = sp.Matrix([
    qw, qx, qy, qz
])

bw = sp.Matrix([
    bwx, bwy, bwz, 
])

ba = sp.Matrix([
    bax, bay, baz
])

dt = sp.symbols('dt')

R = sp.Matrix([
    [1-2*(qy**2 + qz**2), 2*(qx*qy - qw*qz), 2*(qx*qz + qw*qy)],
    [2*(qx*qy - qw*qz), 1-2*(qx**2 + qz**2), 2*(qy*qz-qw*qx)],
    [2*(qx*qz - qw * qy), 2*(qy*qz + qw*qx), 1-2*(qx**2 + qy**2)]
])

ax, ay, az = sp.symbols('a_{x} a_{y} a_{z}')
wx, wy, wz = sp.symbols('w_{x} w_{y} w_{z}')

am = sp.Matrix([
        ax, ay, az
])

wm = sp.Matrix([
        wx, wy, wz
])

g = sp.Matrix([
    0, 0, -9.81
])
 
u = sp.Matrix([
    ax, ay, az, wx, wy, wz
])




In [7]:
v * dt + 0.5*(R*(am - ba) + g)*dt**2 #calculating the position difference (dp)

Matrix([
[        dt**2*(0.5*(a_{x} - b_{ax})*(-2*q_{y}**2 - 2*q_{z}**2 + 1) + 0.5*(a_{y} - b_{ay})*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + 0.5*(a_{z} - b_{az})*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + dt*v_{x}],
[       dt**2*(0.5*(a_{x} - b_{ax})*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + 0.5*(a_{y} - b_{ay})*(-2*q_{x}**2 - 2*q_{z}**2 + 1) + 0.5*(a_{z} - b_{az})*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + dt*v_{y}],
[dt**2*(0.5*(a_{x} - b_{ax})*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + 0.5*(a_{y} - b_{ay})*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + 0.5*(a_{z} - b_{az})*(-2*q_{x}**2 - 2*q_{y}**2 + 1) - 4.905) + dt*v_{z}]])

In [8]:
(R*(am-ba)+g)*dt #calculating the velocity differnce (dv)

Matrix([
[       dt*((a_{x} - b_{ax})*(-2*q_{y}**2 - 2*q_{z}**2 + 1) + (a_{y} - b_{ay})*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + (a_{z} - b_{az})*(2*q_{w}*q_{y} + 2*q_{x}*q_{z}))],
[      dt*((a_{x} - b_{ax})*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + (a_{y} - b_{ay})*(-2*q_{x}**2 - 2*q_{z}**2 + 1) + (a_{z} - b_{az})*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z}))],
[dt*((a_{x} - b_{ax})*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + (a_{y} - b_{ay})*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + (a_{z} - b_{az})*(-2*q_{x}**2 - 2*q_{y}**2 + 1) - 9.81)]])

In [9]:
omega = wm - bw #corrected angular velocity

In [10]:
(omega*dt) #calculating the angular velocity difference at time dt

Matrix([
[dt*(-b_{wx} + w_{x})],
[dt*(-b_{wy} + w_{y})],
[dt*(-b_{wz} + w_{z})]])

In [11]:
   #calculating the rotational differnce delta_q
def delta_q(omega, dt):
    theta = omega.norm() * dt #calculating the total rotation angle
    if theta == 0: #there is no rotation if theta =0
        return sp.Matrix([1, 0, 0, 0])  # nullpöörlemine
    axis = omega / omega.norm() #normalize angular velocity to get rotation axis
    w = sp.cos(theta / 2) # scalar part of quaternion
    xyz = axis * sp.sin(theta / 2) # vector part of quaternion
    return sp.Matrix([w, *xyz]) # return quaternion representing the rotation


In [12]:
dq=delta_q(omega,dt) 

In [13]:
#defining how quaternions are multiplied toghether
def quat_multiply(dq, q):
    qw1, qx1, qy1, qz1 = dq
    qw2, qx2, qy2, qz2 = q
    qw = qw1*qw2 - qx1*qx2 - qy1*qy2 - qz1*qz2
    qx = qw1*qx2 + qx1*qw2 + qy1*qz2 - qz1*qy2
    qy = qw1*qy2 - qx1*qz2 + qy1*qw2 + qz1*qx2
    qz = qw1*qz2 + qx1*qy2 - qy1*qx2 + qz1*qw2
    return sp.Matrix([qw, qx, qy, qz])


In [14]:
# calculating the norm
norm = sp.sqrt(qw**2 + qx**2 + qy**2 + qz**2)

In [15]:
position_t=p+(v * dt + 0.5*(R*(am - ba) + g)*dt**2) #calculating the  position at time t

In [16]:
velocity_t=v+(R*(am-ba)+g)*dt   #calculating the velocity at time t

In [17]:
quaternion_t=quat_multiply(dq, q) #calculating the new quaternion

In [18]:
quaternion_t = quaternion_t / norm # normalize quaternion to unit length

In [19]:
bw_t=bw #gyroscope bias at time t

In [20]:
ba_t=ba #accelometer bias at time t

In [21]:
#Entire drone kinematics equation
f_xu = sp.Matrix.vstack(
    position_t, velocity_t, quaternion_t, bw_t, ba_t
   
)

In [22]:
F=f_xu.jacobian(x) #getting F matrix

In [23]:
G=f_xu.jacobian(u) #getting G matrix

In [25]:
#calculating variance from noise density (ICM-20948)


# gyroscope noise density (0.03 dps/√Hz) → rad/s/√Hz
gyro_noise_density_dps = 0.03
gyro_noise_density_rad = gyro_noise_density_dps * (sp.pi / 180)

In [28]:
# Accelometer noise density (230 µg/√Hz) → m/s²/√Hz
accel_noise_density_ug = 230e-6
accel_noise_density_mps2 = accel_noise_density_ug * 9.80665

In [41]:
# Variance calculation
vg = (gyro_noise_density_rad ** 2) / dt #gyroscope variance
va = (accel_noise_density_mps2 ** 2) / dt #accelometer variance

In [42]:
Q_pos_vel = sp.diag(
    dt**4/4*va**2, dt**4/4*va**2, dt**4/4*va**2,  # pos_x, pos_y, pos_z
    dt**2*va**2, dt**2*va**2, dt**2*va**2           # vel_x, vel_y, vel_z
)

In [46]:
I3 = sp.eye(3)
Q_quat = vg**2 * dt * I3 #calculating the Q matrix for quaternions



In [47]:
sigma_bw = 1e-4    # gyro bias random walk (rad/s²)
sigma_ba = 1e-4    # accel bias random walk (m/s³)

Q_bw = sigma_bw**2 * dt * sp.eye(3)   # gyro bias
Q_ba = sigma_ba**2 * dt * sp.eye(3)   # accel bias


In [49]:
#Calculating the whole Q matrix
Q= sp.diag (
    Q_pos_vel, Q_quat, Q_bw, Q_ba
)

In [50]:
delta_omega=(omega*dt)

In [62]:
h_x = sp.Matrix([
   px, py, pz, vx, vy,vz  # accel osa (lihtsustatud lineaaristatud approximation)
])
h_x

Matrix([
[p_{x}],
[p_{y}],
[p_{z}],
[v_{x}],
[v_{y}],
[v_{z}]])

In [64]:
H=h_x.jacobian(x)
H

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [65]:

P = sp.diag(
    *[.1]*3,     # px, py, pz
    *[0.1]*3,     # vx, vy, vz
    *[0.1]*4,    # qw, qx, qy, qz
    *[0.1]*3,   # bwx, bwy, bwz
    *[0.1]*3    # bax, bay, baz
)
